In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import glob
import numpy as np

from process_results.results_utils import DATASETS, DEFAULT_METRIC, to_dataframe, make_plots, read_log, f

def plot_heatmap(df):
    df["score_str"] =  df.best_metric.map(lambda x: f"{x*100:.1f}\n") + df.last3_mean.map(lambda x: f"{x*100:.1f}") + df.last3_std.map(lambda x: f" ± {x*100:.1f}")
    sns.heatmap(df.pivot("lr","batch_size","best_metric"), annot=df.pivot("lr","batch_size","score_str"),fmt="")
    plt.show()

def logfile_paths(dataset):
    return glob.glob(f"../../checkpoints/finetune/bert_base/{dataset}/**/log",recursive=True)

# for dataset in DATASETS:
#     for logfile in logfile_paths(dataset):
#         df = to_dataframe(logfile, metric=DEFAULT_METRIC[dataset])
#         plot_heatmap(df)

/local/scratch-3/fwe21/miniconda3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# for dataset in DATASETS:
#     score = to_dataframe(logfile_paths(dataset)[0], metric=DEFAULT_METRIC[dataset])["best_metric"].max()
#     print(f"{dataset}: {score}")

In [ ]:
for dataset in DATASETS:
    print(dataset)
    for logfile in glob.glob(f"../../checkpoints/finetune/bert_base/{dataset}/log*"):
        df = to_dataframe(logfile, metric=DEFAULT_METRIC[dataset])
        if "best_metric" in df:
            print(df["best_metric"].max(),logfile)
            make_plots(logfile, metric=DEFAULT_METRIC[dataset])

In [3]:
for dataset in DATASETS:
    plot_repeats(dataset,f"../../checkpoints/finetune/tinybert/{dataset}/log", f"../../checkpoints/finetune/{dataset}/{dataset}/log_rp*")

NameError: name 'plot_repeats' is not defined